# imports

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

# def

In [2]:
def getMoviecode(year=2021, max_page=40):
    
    """연도별 영화코드 수집"""
    
    base_url = 'https://movie.naver.com/movie/sdb/browsing/bmovie.naver?open=' + str(year) + '&page='
    movie_code_list = []
    
    for page in range(max_page + 1):
        
        url = base_url + str(page)
        response = requests.get(url)
        
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'lxml')
        else:
            print('error', response.status_code)
            break
        
        list = soup.select('#old_content > ul > li')
        for movie in range(len(list)):
            code = list[movie].select_one('a').get('href').split('=')[1]
            movie_code_list.append(code)
    
    return movie_code_list

In [10]:
def getSynopsis(code='191559'):
    
    """영화 페이지에서 줄거리 가져오기"""
    
    url = 'https://movie.naver.com/movie/bi/mi/basic.naver?code=' + code
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
    else:
        print('error', response.status_code)
    
    synopsis_list = []
    
    try:
        title   = soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > h3 > a:nth-of-type(1)').text
        content = soup.select_one('#content > div.article > div.section_group.section_group_frst > div:nth-of-type(1) > div > div > p').text
        brief   = soup.select_one('#content > div.article > div.section_group.section_group_frst > div:nth-of-type(1) > div > div > h5').text
        synopsis_list.append(title)
        synopsis_list.append(content)
        synopsis_list.append(brief)
    except:
        synopsis_list.append(np.nan)
        synopsis_list.append(np.nan)
        synopsis_list.append(np.nan)
    
    return synopsis_list

In [11]:
getSynopsis('191559')

['듄',
 "10191년, 아트레이데스 가문의 후계자인 폴(티모시 샬라메)은 시공을 초월한 존재이자\xa0전 우주를 구원할 예지된 자의 운명을 타고났다.\xa0그리고 어떤 계시처럼 매일 꿈에서 아라키스 행성에 있는 한 여인을 만난다.\xa0모래언덕을 뜻하는 '듄'이라 불리는 아라키스는 물 한 방울 없는 사막이지만\xa0우주에서 가장 비싼 물질인 신성한 환각제 스파이스의 유일한 생산지로 이를 차지하기 위한 전쟁이 치열하다.\xa0황제의 명령으로 폴과 아트레이데스 가문은 죽음이 기다리는 아라키스로 향하는데…\xa0\xa0위대한 자는 부름에 응답한다, 두려움에 맞서라, 이것은 위대한 시작이다!",
 '“듄을 지배하는 자가 우주를 지배한다!”']

# 영화 코드 가져오기

In [12]:
movie_codes_2020 = getMoviecode(year=2020, max_page=37)

In [13]:
len(movie_codes_2020), movie_codes_2020[:10]

(750,
 ['195370',
  '193996',
  '194004',
  '187321',
  '194791',
  '194297',
  '182013',
  '194392',
  '189373',
  '195668'])

# 영화 시놉시스 데이터프레임 구성

In [14]:
movie_list_2020 = []
for code in movie_codes_2020:
    movie_list_2020.append(getSynopsis(code=code))

In [ ]:
len(movie_list_2020)

In [ ]:
df = pd.DataFrame(movie_list_2020, columns=['제목', '줄거리', '요약'])
df['개봉년도'] = '2021'
df = df[['제목', '개봉년도', '요약', '줄거리']]

In [ ]:
df['요약'] = df['요약'].str.replace('\r', ' ')
df['요약'] = df['요약'].str.replace('\n', '')
df['요약'] = df['요약'].str.replace('\xa0', '')

df['줄거리'] = df['줄거리'].str.replace('\r', ' ')
df['줄거리'] = df['줄거리'].str.replace('\n', '')
df['줄거리'] = df['줄거리'].str.replace('\xa0', '')

In [ ]:
df.head(100)

In [ ]:
df.to_csv('./movie_synopsis_2020.csv')